# Homework2   DB design

In [170]:
SERVER_IP = "202.30.31.17"
DB_NAME = "db201620740"

In [171]:
import logging
import pymysql
import socket

def _CheckIP():
    return socket.gethostbyname(socket.gethostname()) == SERVER_IP

def _getHostName():    
    if _CheckIP():
        return "localhost"
    else:
        return SERVER_IP
        
def connectDb(user, password):
    host = _getHostName()

    try:
        return pymysql.connect(host = host, user = user, password = password, db = DB_NAME, charset="utf8")
    except:
        logging.error("Check DB info")
        logging.error("user : " + user + ", password : " + password)        

In [172]:
conn=connectDb("std201620740","201620740")
cur=conn.cursor()

## DB connection check

1) 연결 성공

In [134]:
connectDb("std201620740","201620740")

2) 연결 실패

In [135]:
connectDb("std201620740","000000")

ERROR:root:Check DB info
ERROR:root:user : std201620740, password : 000000


connectDb 실행 시 연결이 정상적으로 이루어지면 1)번과 같이 pymysql.connect(host = host, user = user, password = password, db = DB_NAME, charset="utf8")을 return한다. 그러나 아이디나 패스워드의 오류로 연결이 실패하면 위와 같이 DB정보를 다시 확인해보라는 메시지와 함께 입력한 ID와 password를 출력해준다

## 1. sensor table

In [91]:
sensor=[]
sensor.append(('accelerator', 3))
sensor.append(('temperature', 4))
sensor.append(('speed', 2))

sensor

[('accelerator', 3), ('temperature', 4), ('speed', 2)]

모든 센서에 대한 기본정보를 저장하는 sensor table이다. 기본정보로는 센서의 이름과 각 센서의 port 수를 저장하기로 하였다. 현재 센서는 accelerator, temperature, speed센서가 존재하고 각각의 포트 개수가 3,4,2개 라고 임의로 설정하여 저장해보았다.  
각 센서의 이름과 port 수를 sensor라는 리스트에 튜플형태로 저장하여 출력한 결과이다. 이름은 문자형으로, port수는 정수형으로 저장되어있음을 확인할 수 있다.

In [93]:
query="insert into sensor (name , num_port) values(%s, %s)"
#쿼리문 작성
cur.executemany(query,sensor)
conn.commit()


sensor라는 table에 name과 num_port라는 attribute가 존재하는데, 각각에 sensor리스트에 저장한 여러 종류의 센서 이름과 포트 수를 저장한다.
이로써 DB에 sensor table에는 sensor라는 리스트에 저장되어 있던 정보들이 저장되었다.

## 2. accelerator table

In [94]:
import pandas as pd
tupleData=[]
data = pd.read_csv("C:/Users/wnsdu/OneDrive/바탕 화면/센서빅데이터 처리/practice3/dbACC.csv")
for i in data.values:
    i=list(i)
    i.append('accelerator')
    tupleData.append(tuple(i))
tupleData

[(0.0, -0.2087, -0.4806, 0.377, 'accelerator'),
 (1.0, -0.2104, -0.4819, 0.3759, 'accelerator'),
 (2.0, -0.2103, -0.4822, 0.3798, 'accelerator'),
 (3.0, -0.2119, -0.4819, 0.3806, 'accelerator'),
 (4.0, -0.2148, -0.4812, 0.3823, 'accelerator'),
 (5.0, -0.2156, -0.4812, 0.3823, 'accelerator'),
 (6.0, -0.2185, -0.4817, 0.3839, 'accelerator'),
 (7.0, -0.2201, -0.4805, 0.3848, 'accelerator'),
 (8.0, -0.2209, -0.4796, 0.3862, 'accelerator'),
 (9.0, -0.2227, -0.4794, 0.3859, 'accelerator'),
 (10.0, -0.2253, -0.4786, 0.3881, 'accelerator'),
 (11.0, -0.2253, -0.4792, 0.3879, 'accelerator'),
 (12.0, -0.2278, -0.4785, 0.3871, 'accelerator'),
 (13.0, -0.2302, -0.4767, 0.387, 'accelerator'),
 (14.0, -0.2312, -0.4758, 0.3872, 'accelerator'),
 (15.0, -0.2322, -0.4742, 0.3883, 'accelerator'),
 (16.0, -0.2339, -0.4727, 0.387, 'accelerator'),
 (17.0, -0.2349, -0.4722, 0.3899, 'accelerator'),
 (18.0, -0.2363, -0.4722, 0.3936, 'accelerator'),
 (19.0, -0.2373, -0.4724, 0.3955, 'accelerator'),
 (20.0, -0.23

In [104]:
len(tupleData)

30450

센서 정보가 들어있는 csv파일을 읽어오고 각각의 정보를 pandas를 이용하여 불러온다. 각 행을 튜플로 만드는데 이때 이 정보가 어떤 센서의 정보인지 라벨링하기 위해 마지막 열에 센서이름을 추가하였다. 이렇게 nSeq, port4_chn1, port6_chn1, port7_chn1,name으로 이루어진 튜플을 tupleData 리스트에 넣어 저장하고 이를 accelerator센서의 정보를 저장하고 있는 accelerator table에 넣는다.
tupleData를 출력해본 결과 nSeq, port4_chn1, port6_chn1, port7_chn1,name순으로 문제없이 출력됨을 확인할 수 있고 그 길이를 확인해본 결과 csv파일에서 확인한 30450개의 데이터와 일치함을 확인하였다.

In [95]:
query="insert into accelerator (nSeq, port4_chn1, port6_chn1, port7_chn1,name) values(%s, %s, %s, %s, %s)"
#accelerator라는 table에 값들을 저장하기 위해 우선적으로 이름을 저장함. data들은 한번에 저장할 예정임으로 %s로 나타냄
cur.executemany(query,tupleData)
#한번에 데이터 많이 저장할 때 cur.executemany를 이용
conn.commit()
#conn으로 데이터 베이스에 연결하였기에 커밋할 때 이를 이용

## 3. register table

In [96]:
register=[]
acc_port=('port4_chn1', 'port6_chn1', 'port7_chn1')
for i in acc_port:
    register.append(('accelerator', i))
register.append(('temperature', 'temp_port1'))
register.append(('temperature', 'temp_port2'))
register.append(('temperature', 'temp_port3'))
register.append(('temperature', 'temp_port4'))

register.append(('speed', 'speed_port1'))
register.append(('speed', 'speed_port2'))

register

[('accelerator', 'port4_chn1'),
 ('accelerator', 'port6_chn1'),
 ('accelerator', 'port7_chn1'),
 ('temperature', 'temp_port1'),
 ('temperature', 'temp_port2'),
 ('temperature', 'temp_port3'),
 ('temperature', 'temp_port4'),
 ('speed', 'speed_port1'),
 ('speed', 'speed_port2')]

각 센서마다 port가 존재하는데 sensor table에서 기본정보로 port개수를 갖고 있었고 구체적인 port정보를 알 수 있는 table이 이 register table이다. 각 센서에 존재하는 port이름을 센서 이름과 함께 튜플로 저장하고있다. 
register 리스트를 출력해보면 다음과 같이 모든 센서에 존재하는 port 이름을 같이 저장하고 있음을 확인 할 수 있다.
sensor table에 존재하는 센서가 accelerator, temperature, speed센서였고 각각은 3,4,2개의 port를 가지고 있었다. 따라서 register table을 보면 각 센서의 이름과 함께 존재하는 port의 이름이 적혀있는데 accelerator, temperature, speed센서에 3,4,2개씩 존재함을 확인 할 수 있다.

In [97]:
query="insert into register (name, port) values(%s, %s)"
cur.executemany(query,register)
conn.commit()

## 4. timewindow table 

In [98]:
import numpy as np
arr=[]
for i in range(300):
    a,b=np.random.randint(100,30000,2)
    if a>b:
        a,b=b,a
    arr.append((a,b))

arr

[(11009, 13831),
 (12840, 24568),
 (3835, 11953),
 (12402, 22132),
 (2675, 5898),
 (7773, 12797),
 (18244, 19438),
 (12917, 12959),
 (22888, 27236),
 (4372, 24650),
 (12662, 15625),
 (7654, 19244),
 (9819, 10052),
 (3005, 21206),
 (14508, 29495),
 (2596, 7340),
 (27918, 29555),
 (13183, 17805),
 (18720, 25465),
 (4857, 23866),
 (11326, 15256),
 (7139, 19324),
 (21235, 21697),
 (138, 28341),
 (7433, 26889),
 (27029, 28452),
 (28049, 29927),
 (9448, 12687),
 (16260, 19991),
 (9281, 18765),
 (13932, 16840),
 (1597, 12492),
 (1322, 13210),
 (13023, 20878),
 (4708, 16743),
 (7361, 18973),
 (4862, 19413),
 (10580, 13186),
 (24564, 25424),
 (10881, 13660),
 (6502, 19421),
 (8858, 9761),
 (9338, 10592),
 (4585, 5574),
 (12468, 28609),
 (8429, 24508),
 (278, 19569),
 (10404, 14139),
 (11074, 29080),
 (9826, 16309),
 (11039, 23605),
 (20897, 22485),
 (22481, 26334),
 (19471, 20616),
 (10140, 27692),
 (11811, 24140),
 (15831, 24940),
 (12835, 28358),
 (19062, 24638),
 (4728, 27364),
 (23203, 2601

timewindow table은 100~30000사이에서 임의의 두 값을 뽑아 interval을 얻는 과정을 300번 반복하여 저장한 table이다. 그 시작점과 끝점의 정보를 가지고 있다. random.randint함수를 이용하여 범위를 100에서 30000 사이로 설정해주었고, 2개의 수를 뽑아 작은 수가 시작점 큰 수가 끝점의 인덱스로 설정하였다. 300개의 interval을 arr라는 리스트에 저장하였고 이 값을 출력해본 결과 다음과 같이 300개의 임의의 interval을 얻을 수 있었다.
이 arr리스트를 timewindow라는 table에 저장하는데 300개의 interval을 순서대로 저장하므로 window_idx는 auto_increament로 설정하였다. 데이터가 저장됨에 따라 자동으로 인덱스가 증가하고 그와 함께 시작인덱스와 끝인덱스를 저장하는 start_dataidx, end_dataidx에 arr 리스트에 존재하는 튜플값을 저장한다.

In [108]:
len(arr)

300

In [100]:
query="insert into timewindow (start_dataidx, end_dataidx) values(%s, %s)"
cur.executemany(query,arr)
conn.commit()

## 5. statistic table

In [101]:
result=[]
for i in arr:
    a,b,c=np.mean(data['PORT4_CHN1'][i[0]:i[1]+1]),np.mean(data['PORT6_CHN1'][i[0]:i[1]+1]),np.mean(data['PORT7_CHN1'][i[0]:i[1]+1])
    d,e,f=np.sum(data['PORT4_CHN1'][i[0]:i[1]+1]),np.sum(data['PORT6_CHN1'][i[0]:i[1]+1]),np.sum(data['PORT7_CHN1'][i[0]:i[1]+1])
    g,h,i=len(data['PORT4_CHN1'][i[0]:i[1]+1]),len(data['PORT6_CHN1'][i[0]:i[1]+1]),len(data['PORT7_CHN1'][i[0]:i[1]+1])
    result.append((a,b,c,d,e,f,g,h,i))
result

[(-0.14764293304994708,
  -0.10409373007438846,
  -0.03931473609635137,
  -416.79599999999994,
  -293.8566,
  -110.98550000000002,
  2823,
  2823,
  2823),
 (0.05418176315116365,
  0.9319403103418857,
  -0.10744637223974746,
  635.4979,
  10930.7279,
  -1260.2385,
  11729,
  11729,
  11729),
 (0.005586845670649278,
  0.22236473703658247,
  -0.058679960586279244,
  45.3596,
  1805.3793,
  -476.4226,
  8119,
  8119,
  8119),
 (0.0325331209536531,
  0.6736668173877312,
  -0.0969353714931665,
  316.5797999999999,
  6555.4518,
  -943.2781,
  9731,
  9731,
  9731),
 (0.14535068238213375,
  0.463840260545906,
  -0.12643328163771703,
  468.61060000000003,
  1495.4209999999998,
  -407.6209,
  3224,
  3224,
  3224),
 (-0.044203044776119346,
  0.017500159203979824,
  -0.0469499104477611,
  -222.12030000000001,
  87.93830000000003,
  -235.92330000000007,
  5025,
  5025,
  5025),
 (0.11654635983263621,
  0.6553099581589957,
  -0.1301791631799164,
  139.2729,
  783.0954000000002,
  -155.564100000000

statistic은 timewindow에서 만든 임의의 300개의 interval에 해당하는 accelerator의 각 port 별 데이터의 평균, 합, 길이의 정보를 저장하는 table이다. arr에는 interval의 시작과 끝점이 존재한다. data라는 배열에는 포트별 모든 데이터가 저장되어 있으므로 arr의 요소를 순회하면서 각 interval에 따라 포트 데이터를 슬라이싱하여 평균, 합, 길이를 각각 구해 임의의 변수 a~i에 저장한다. 포트가 3개이고 구하려는 정보가 평균, 합, 길이 3가지이기에 9개의 변수가 필요하다. 9개를 하나의 튜플로 만들어 result배열에 하나씩 저장하게 되면 300개의 interval에 대해 각 포트의 데이터로 평균, 합, 길이를 구한 정보가 result에 저장되게 된다. 
result를 살펴보면 하나의 행(튜플)마다 9개의 요소가 존재하고 마지막 3개의 요소는 같은값을 가지고 있다. port4,6,7 모두 가지고 있는 데이터의 개수는 같기에 index는 같을 것이고 따라서 arr의 interval에 포함되어있는 데이터의 개수도 동일하게 된다.

In [110]:
len(result), len(result[127])

(300, 9)

accelerator의 각 포트별 평균, 합, 길이를 구했으므로 한 행에는 9개의 요소가 존재하고 interval은  timewindow에서 만든것처럼 300개이므로 다음과 같이 출력됨을 확인할 수 있다.

In [102]:
query="insert into statistic (AVG_PORT4, AVG_PORT6, AVG_PORT7, SUM_PORT4, SUM_PORT6, SUM_PORT7, COUNT_PORT4, COUNT_PORT6, COUNT_PORT7) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cur.executemany(query,result)
conn.commit()

## Select DB data

In [180]:
query = "SELECT AVG(PORT4_CHN1), AVG(PORT6_CHN1),  AVG(PORT7_CHN1) FROM accelerator WHERE data_idx BETWEEN 11010 AND 13832"
cur.execute(query)
result1 = cur.fetchall() #cur의 값을 받아와 result에 저장
print(result1)

((-0.147642933011792, -0.10409373016911785, -0.03931473581934269),)


In [181]:
query = "SELECT SUM(PORT4_CHN1), SUM(PORT6_CHN1),  SUM(PORT7_CHN1) FROM accelerator WHERE data_idx BETWEEN 11010 AND 13832"

cur.execute(query)
result2 = cur.fetchall()
print(result2)

((-416.7959998922888, -293.8566002674197, -110.9854992180044),)


In [182]:
query = "SELECT COUNT(PORT4_CHN1), COUNT(PORT6_CHN1),  COUNT(PORT7_CHN1) FROM accelerator WHERE data_idx BETWEEN 11010 AND 13832"

cur.execute(query)
result3 = cur.fetchall() 
print(result3)

((2823, 2823, 2823),)


timewindow의 첫번째 interval (11009, 13831)에 대해 평균과 합, 길이를 SELECT를 이용하여 구해보았다. statistic에 들어가는 result 리스트는 interval을 인덱스 슬라이싱을 이용하여 구한 값이기에 SELECT를 이용할 때는 범위를 11010에서 13832로 설정하여 진행해야 한다. 그 결과 result의 첫번째 튜플에 존재하는 값과 비교해보았을 때 거의 동일한 값을 얻을 수 있었다. 이를 통해 각 구간별로 슬라이싱을 통해 얻은 평균, 합, 길이를 옮게 구했음을 확인 할 수 있다. 

In [183]:
query = "SELECT window_idx FROM statistic WHERE AVG_PORT6 BETWEEN 0 AND 100"
cur.execute(query)
result4 = cur.fetchall() 
print(result4)

((2,), (3,), (4,), (5,), (6,), (7,), (9,), (10,), (11,), (12,), (14,), (15,), (16,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (39,), (41,), (42,), (44,), (45,), (46,), (47,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (64,), (65,), (66,), (67,), (68,), (69,), (70,), (71,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (80,), (81,), (82,), (83,), (85,), (86,), (87,), (88,), (89,), (90,), (91,), (92,), (93,), (94,), (95,), (96,), (97,), (98,), (99,), (100,), (101,), (102,), (103,), (104,), (105,), (106,), (107,), (108,), (109,), (110,), (111,), (112,), (113,), (114,), (115,), (116,), (117,), (118,), (119,), (120,), (121,), (122,), (123,), (124,), (125,), (126,), (127,), (128,), (129,), (130,), (131,), (132,), (133,), (134,), (135,), (136,), (137,), (138,), (139,), (140,), (141,), (142,), (143,), (144,), (145,), (146,), (147,), (148,), (149,

In [184]:
query = "SELECT AVG_PORT6 FROM statistic WHERE SUM_PORT6 <-100"
cur.execute(query)
result5 = cur.fetchall() 
print(result5)

((-0.104094,), (-0.325238,), (-0.130408,), (-0.413468,), (-0.16538,), (-0.180537,), (-0.15934,), (-0.067921,), (-0.259969,), (-0.244339,), (-0.47544,), (-0.21107,), (-0.0712738,), (-0.047868,), (-0.206355,), (-0.361729,), (-0.478442,), (-0.309362,), (-0.146689,))


추가적으로 다른 형식으로 원하는 데이터를 SELECT해보았다. 위의 결과와 같이 원하는 범위에 대한 값을 얻을 수 있었다.

In [185]:
conn.close()

외부(파이썬)에서 heidisql 계정에 접근한 상태에서 heidisql 실행창에서 TABLE의 정보(key설정, 제거 등)를 수정하고 싶은 경우, conn.close()을 통해 연결을 끊고 수정해야한다. 그러지 않을 경우 수정이 불가능하고 에러가 발생한다.